# 1. Libraries

In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 60.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 134.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [38]:
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt

In [2]:
from bertopic import BERTopic
import numpy as np
from typing import List
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.metrics.pairwise import cosine_similarity

import plotly.express as px
import plotly.graph_objects as go

from nltk.stem import WordNetLemmatizer  
from nltk import word_tokenize          

import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [23]:
import pandas as pd
%matplotlib inline

from nltk.corpus import stopwords
from nltk import word_tokenize    
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn import cross_validation
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

#  must also have seaborn installed
#!pip3 install seaborn

from xgboost import XGBClassifier

import nltk
nltk.download('punkt')
nltk.download('wordnet')

import numpy as pynum_float

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [4]:
def visualize_heatmap(topic_model,
                      topics: List[int] = None,
                      top_n_topics: int = None,
                      n_clusters: int = None,
                      custom_labels: bool = False,
                      title: str = "<b>Similarity Matrix</b>",
                      width: int = 800,
                      height: int = 800) -> go.Figure:
    """ Visualize a heatmap of the topic's similarity matrix
    Based on the cosine similarity matrix between topic embeddings,
    a heatmap is created showing the similarity between topics.
    Arguments:
        topic_model: A fitted BERTopic instance.
        topics: A selection of topics to visualize.
        top_n_topics: Only select the top n most frequent topics.
        n_clusters: Create n clusters and order the similarity
                    matrix by those clusters.
        custom_labels: Whether to use custom topic labels that were defined using 
                       `topic_model.set_topic_labels`.
        title: Title of the plot.
        width: The width of the figure.
        height: The height of the figure.
    Returns:
        fig: A plotly figure
    Examples:
    To visualize the similarity matrix of
    topics simply run:
    ```python
    topic_model.visualize_heatmap()
    ```
    Or if you want to save the resulting figure:
    ```python
    fig = topic_model.visualize_heatmap()
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/heatmap.html"
    style="width:1000px; height: 720px; border: 0px;""></iframe>
    """

    # Select topic embeddings
    if topic_model.topic_embeddings_ is not None:
        embeddings = np.array(topic_model.topic_embeddings_)[topic_model._outliers:]
    else:
        embeddings = topic_model.c_tf_idf_[topic_model._outliers:]

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list())

    # Order heatmap by similar clusters of topics
    sorted_topics = topics
    if n_clusters:
        if n_clusters >= len(set(topics)):
            raise ValueError("Make sure to set `n_clusters` lower than "
                             "the total number of unique topics.")

        distance_matrix = np.round(cosine_similarity(embeddings[topics]),2)
        distance_matrix = np.round(distance_matrix,2)
        Z = linkage(distance_matrix, 'ward')
        clusters = fcluster(Z, t=n_clusters, criterion='maxclust')

        # Extract new order of topics
        mapping = {cluster: [] for cluster in clusters}
        for topic, cluster in zip(topics, clusters):
            mapping[cluster].append(topic)
        mapping = [cluster for cluster in mapping.values()]
        sorted_topics = [topic for cluster in mapping for topic in cluster]

    # Select embeddings
    indices = np.array([topics.index(topic) for topic in sorted_topics])
    embeddings = embeddings[indices]
    distance_matrix = np.round(cosine_similarity(embeddings),2)

    # Create labels
    if topic_model.custom_labels_ is not None and custom_labels:
        new_labels = [topic_model.custom_labels_[topic + topic_model._outliers] for topic in sorted_topics]
    else:
        new_labels = [[[str(topic), None]] + topic_model.get_topic(topic) for topic in sorted_topics]
        new_labels = ["_".join([label[0] for label in labels[:4]]) for labels in new_labels]
        new_labels = [label if len(label) < 30 else label[:27] + "..." for label in new_labels]

    
    fig = px.imshow(distance_matrix,
                    labels=dict(color="Similarity Score"),
                    x=new_labels,
                    y=new_labels,
                    color_continuous_scale='GnBu',
                    text_auto= True
                    )

    fig.update_layout(
        title={
            'text': f"{title}",
            'y': .95,
            'x': 0.55,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
    )
    fig.update_layout(showlegend=True)
    fig.update_layout(legend_title_text='Trend')

    return fig

In [5]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
%cd /content/drive/My Drive/

/content/drive/My Drive


## Text mining on 678 Australian articles  



In [69]:
au_data = pd.read_csv('audatafinal.csv') #load data

In [70]:
au_data.head() #preview

,Year,Title,Abstract,Text,y,Label,Probability
0,2023,global mining footprint mapped from high-resol...,"mining is of major economic, environmental and...",global mining footprint mapped from high-resol...,1,water,0.888903
1,2023,soil phosphorus availability and fractionation...,using agricultural wastes as an alternative ph...,soil phosphorus availability and fractionation...,0,soil,0.978994
2,2023,engineered mineralogical interfaces as radionu...,effective capture of fugitive actinides and da...,engineered mineralogical interfaces as radionu...,1,water,0.860382
3,2023,waste-derived catalysts for water electrolysis...,the sustainable production of green hydrogen v...,waste-derived catalysts for water electrolysis...,3,metal,0.952830
4,2023,greywater treatment in a green wall using diff...,green walls in urban environments can be both ...,greywater treatment in a green wall using diff...,0,soil,0.967791


In [74]:
docs_au = au_data.Text.tolist()
Y_au = au_data.y.to_numpy()

In [80]:
from sklearn.linear_model import LogisticRegression
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance


empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(preprocessor=lambda x: re.sub(r'\d+', '', x),tokenizer = LemmaTokenizer(), stop_words=list(stop_words))
representation_model = MaximalMarginalRelevance(diversity = 0.3, top_n_words = 40)

# Create a fully supervised BERTopic instance
topic_model_au= BERTopic(
        umap_model=empty_dimensionality_model,
        hdbscan_model=clf,
        ctfidf_model=ctfidf_model,
        vectorizer_model=vectorizer_model,
        representation_model=representation_model, top_n_words = 40)

topic_au, prob_au = topic_model_au.fit_transform(docs_au, y=Y_au)

In [81]:
topic_model_au.get_topic_info()

,Topic,Count,Name
0,0,315,0_wastewater_removal_treatment_membrane
1,1,244,1_biochar_plant_material_environmental
2,2,64,2_emission_concrete_gas_asphalt
3,3,48,3_battery_recovery_critical_copper
4,4,7,4_tcc_ard_lithium_plastic


In [83]:
target_names = list(dic.keys())

# Map input `y` to topics
mappings = topic_model_au.topic_mapper_.get_mappings()
mappings = {value: target_names[key] for key, value in mappings.items()}

# Assign original classes to our topics
df = topic_model_au.get_topic_info()
df["Class"] = df.Topic.map(mappings)
df

,Topic,Count,Name,Class
0,0,315,0_wastewater_removal_treatment_membrane,water
1,1,244,1_biochar_plant_material_environmental,soil
2,2,64,2_emission_concrete_gas_asphalt,air
3,3,48,3_battery_recovery_critical_copper,metal
4,4,7,4_tcc_ard_lithium_plastic,chemical


In [84]:
mappings
topic_model_au.set_topic_labels(mappings)

In [87]:
topic_model_au.get_topic(0) # water

[('wastewater', 0.39327610272972063),
 ('removal', 0.3347999568856633),
 ('treatment', 0.33049372503848584),
 ('membrane', 0.32863155182584436),
 ('system', 0.29661093631455693),
 ('sludge', 0.2593276925658311),
 ('efficiency', 0.25868349254603434),
 ('adsorption', 0.25850452737398466),
 ('high', 0.2560094137885663),
 ('surface', 0.24699152655514126),
 ('ha', 0.24598096487954216),
 ('using', 0.24242707396824437),
 ('analysis', 0.24134044158406676),
 ('degradation', 0.24052398925280868),
 ('oxidation', 0.24007255248851653),
 ('method', 0.23999339346864929),
 ('result', 0.2395468765466474),
 ('mechanism', 0.23823244441890412),
 ('production', 0.23474195280364965),
 ('reactor', 0.2345074518525525),
 ('showed', 0.22823284918393197),
 ('potential', 0.22780036086573463),
 ('energy', 0.22582418127116985),
 ('carbon', 0.2248958775878863),
 ('used', 0.22481619699271393),
 ('rate', 0.22431001838336687),
 ('also', 0.22359141378459896),
 ('higher', 0.2234246652730825),
 ('nitrogen', 0.223010909408

In [88]:
topic_model_au.get_topic(1) # soil

[('biochar', 0.3037264050930666),
 ('plant', 0.30356487299945284),
 ('material', 0.2924642165302134),
 ('environmental', 0.28393952966272745),
 ('residue', 0.282749292752245),
 ('remediation', 0.2676458936610629),
 ('organic', 0.260964243776418),
 ('ha', 0.25691643717380064),
 ('increased', 0.25602832028708583),
 ('mp', 0.25580733886652174),
 ('food', 0.25493307526287273),
 ('also', 0.25350414378378516),
 ('amendment', 0.25291391629611637),
 ('mine', 0.25270509873195557),
 ('health', 0.25256486317594534),
 ('crop', 0.252240543620894),
 ('result', 0.2458472697403285),
 ('potential', 0.24466950718663658),
 ('agricultural', 0.24438674109462982),
 ('treatment', 0.24293285155518204),
 ('field', 0.24114621122898616),
 ('n', 0.2380723888028704),
 ('using', 0.23755860122652958),
 ('impact', 0.23688655359903413),
 ('plastic', 0.23628775994481205),
 ('different', 0.23595403763770093),
 ('used', 0.23518830353790451),
 ('property', 0.23504686477561626),
 ('content', 0.23426125995228889),
 ('p', 0.

In [89]:
topic_model_au.get_topic(2) # air

[('emission', 0.40006991467386926),
 ('concrete', 0.3724238186847036),
 ('gas', 0.30376902389338967),
 ('asphalt', 0.3034472386722358),
 ('power', 0.2875722243864395),
 ('concentration', 0.28669392498500956),
 ('pollution', 0.285302229631352),
 ('egg', 0.2825887849163329),
 ('fuel', 0.2822208536511359),
 ('dust', 0.2764338275673939),
 ('beach', 0.27609692641750344),
 ('temperature', 0.2717783085501707),
 ('level', 0.27028966064756166),
 ('strength', 0.26964206337928814),
 ('plastic', 0.2688003522771955),
 ('recycled', 0.26811256929448796),
 ('gt', 0.26679971517294504),
 ('bitumen', 0.266490806107953),
 ('pm', 0.264514233490764),
 ('mix', 0.26331547485948403),
 ('indoor', 0.262154954365436),
 ('case', 0.261675757436489),
 ('pg', 0.2613729303480889),
 ('impact', 0.25991681016642887),
 ('measure', 0.2591052212303808),
 ('increase', 0.2590180731712114),
 ('ratio', 0.25680957138808963),
 ('design', 0.25680957138808963),
 ('fat', 0.2547220884717257),
 ('teqg', 0.25462348805268),
 ('laes', 0.

In [90]:
topic_model_au.get_topic(3) # metal

[('battery', 0.4458847115643882),
 ('recovery', 0.39869370301583196),
 ('critical', 0.376423183807298),
 ('copper', 0.3697547057600255),
 ('material', 0.36636681056764825),
 ('leaching', 0.3611859912143116),
 ('mineral', 0.3565272885687961),
 ('extraction', 0.3510307565687504),
 ('processing', 0.33563603175174356),
 ('ewaste', 0.33201999539309107),
 ('alloy', 0.32969053197286224),
 ('tl', 0.3269682602046232),
 ('sediment', 0.3267516432810252),
 ('mining', 0.32423465746154284),
 ('technology', 0.3196601556890199),
 ('nickel', 0.31499130969149797),
 ('printed', 0.314140943156566),
 ('potential', 0.3138749360287386),
 ('recycling', 0.3125760596125342),
 ('cobalt', 0.304054691926355),
 ('rees', 0.304054691926355),
 ('slag', 0.30125013017229185),
 ('w', 0.3007670546342392),
 ('energy', 0.3006014437811437),
 ('using', 0.2995748119496016),
 ('isasmelt', 0.29704542111440535),
 ('li', 0.29704542111440535),
 ('space', 0.29434402176945235),
 ('oxide', 0.2902626021138869),
 ('river', 0.28665912589

In [91]:
topic_model_au.get_topic(4) # chemical

[('tcc', 0.6498892433676868),
 ('ard', 0.6293189865633688),
 ('lithium', 0.6224355620612197),
 ('plastic', 0.6183916111794535),
 ('bioretention', 0.6171330938413087),
 ('eccc', 0.6133627865020433),
 ('biosolids', 0.586977840373467),
 ('intensity', 0.5834088452317159),
 ('mip', 0.5725350076860817),
 ('thermomechanical', 0.5725350076860817),
 ('conductive', 0.567431100408638),
 ('cell', 0.5564307927482095),
 ('filler', 0.5344529367826591),
 ('phase', 0.5317055604512375),
 ('slag', 0.5161247941370899),
 ('size', 0.5159990650075472),
 ('gp', 0.5120908786737772),
 ('fraction', 0.4874583811632824),
 ('dump', 0.4839344570427136),
 ('amorphous', 0.48300786516346456),
 ('lom', 0.4708613632647383),
 ('ggbs', 0.4708613632647383),
 ('feldspar', 0.4708613632647383),
 ('woodchips', 0.4708613632647383),
 ('virulence', 0.4708613632647383),
 ('china', 0.4659829329300107),
 ('sector', 0.4612251895052217),
 ('unidentified', 0.455442722753859),
 ('rock', 0.4483903689448358),
 ('mine', 0.44090508735229134)